In [1]:
import os 
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=os.environ['GITHUB_API_TOKEN'],
    )

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name

)



In [5]:
response.choices[0].message.content

'The capital of England is **London**.'

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

### Simple chain

In [10]:
model = "gpt-4o"

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}."
)

model = ChatOpenAI(api_key=openai_api_key,temperature=0.0, model=model)
outputParser = StrOutputParser()

In [11]:
chain = prompt | model | outputParser

In [12]:
chain.invoke({"topic": "cats"})

"Sure! Here's a cat joke for you:\n\nWhy was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse! 🐱🖱️"